# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt','wordnet','stopwords'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# import libraries
import re
import time
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('disaster_data1',engine)
df.head()


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26345 entries, 0 to 26344
Data columns (total 40 columns):
id                        26345 non-null int64
message                   26345 non-null object
original                  10223 non-null object
genre                     26345 non-null object
related                   26207 non-null float64
request                   26207 non-null float64
offer                     26207 non-null float64
aid_related               26207 non-null float64
medical_help              26207 non-null float64
medical_products          26207 non-null float64
search_and_rescue         26207 non-null float64
security                  26207 non-null float64
military                  26207 non-null float64
child_alone               26207 non-null float64
water                     26207 non-null float64
food                      26207 non-null float64
shelter                   26207 non-null float64
clothing                  26207 non-null float64
money         

In [5]:
df.isnull().sum()


id                            0
message                       0
original                  16122
genre                         0
related                     138
request                     138
offer                       138
aid_related                 138
medical_help                138
medical_products            138
search_and_rescue           138
security                    138
military                    138
child_alone                 138
water                       138
food                        138
shelter                     138
clothing                    138
money                       138
missing_people              138
refugees                    138
death                       138
other_aid                   138
infrastructure_related      138
transport                   138
buildings                   138
electricity                 138
tools                       138
hospitals                   138
shops                       138
aid_centers                 138
other_in

In [6]:
df=df.dropna()
df.isnull().sum()

id                        0
message                   0
original                  0
genre                     0
related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                

In [7]:
X = df['message'].values

y = df.iloc[:,4:].values





### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex,text)
    for url in detected_urls:
        text =text.replace(url,'urlplaceholder')
    
    tokens = [ lemmatizer.lemmatize(word).strip() for word in word_tokenize(text.lower()) if word.isalnum() and word not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
    ('vect_count', CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

#### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train,X_test,y_train,y_test= train_test_split(X,y)

In [11]:
#train classifier
pipeline.fit(X_train,y_train)
#y_pred= pipeline.predict(X_test)

Pipeline(memory=None,
     steps=[('vect_count', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
def model_performance(model,X_test,y_test):
    categories_names=list(df[4:])
    y_pred=model.predict(X_test)
    for i in range(len(categories_names)):
        print("-------------------- \n",
             "categories_name {}",format(categories_names[i]),
             "\n",
             classification_report(y_test[:, i],y_pred[:, i]))

In [13]:
model_performance(pipeline,X_test,y_test)

-------------------- 
 categories_name {} id 
              precision    recall  f1-score   support

        0.0       0.37      0.26      0.30       847
        1.0       0.67      0.78      0.72      1680
        2.0       0.33      0.03      0.06        29

avg / total       0.57      0.60      0.58      2556

-------------------- 
 categories_name {} message 
              precision    recall  f1-score   support

        0.0       0.65      0.88      0.75      1647
        1.0       0.41      0.15      0.22       909

avg / total       0.56      0.62      0.56      2556

-------------------- 
 categories_name {} original 
              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2553
        1.0       0.00      0.00      0.00         3

avg / total       1.00      1.00      1.00      2556

-------------------- 
 categories_name {} genre 
              precision    recall  f1-score   support

        0.0       0.63      0.85      0.72    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


IndexError: index 36 is out of bounds for axis 1 with size 36

### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect_count',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f4f26b1af28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state

In [14]:

parameters = {
    'vect_count__ngram_range': ((1, 1),(1,2)),
    'vect_count__max_df': (0.5,0.75,1.0),
    'tfidf__use_idf': (True,False),
    'clf__estimator__n_estimators': [10,20]
             }
start= time.process_time()

cv = GridSearchCV(estimator=pipeline, param_grid= parameters, n_jobs= 4, verbose=2)
cv.fit(X_train, y_train)
print(time.process_time() - start)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_range=(1, 1) 
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_range=(1, 1) 
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_range=(1, 1) 
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_range=(1, 1), total=  57.9s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_range=(1, 1), total=  58.4s
[CV] clf__estimator__n_estimators=10, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_ran

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 13.7min


[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect_count__max_df=1.0, vect_count__ngram_range=(1, 2), total= 1.7min
[CV] clf__estimator__n_estimators=20, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect_count__max_df=1.0, vect_count__ngram_range=(1, 2), total= 1.6min
[CV] clf__estimator__n_estimators=20, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=20, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_range=(1, 1), total= 1.6min
[CV] clf__estimator__n_estimators=20, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=10, tfidf__use_idf=False, vect_count__max_df=1.0, vect_count__ngram_range=(1, 2), total= 1.7min
[CV] clf__estimator__n_estimators=20, tfidf__use_idf=True, vect_count__max_df=0.5, vect_count__ngram_range=(1, 2) 
[CV]  clf__estima

[Parallel(n_jobs=4)]: Done  72 out of  72 | elapsed: 38.9min finished


82.741066091


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
pipeline2 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),

    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])


pipeline2.fit(X_train, y_train)
 
model_performance(pipeline2,X_test, y_test)

-------------------- 
 categories_name {} id 
              precision    recall  f1-score   support

        0.0       0.33      0.42      0.37       847
        1.0       0.66      0.58      0.62      1680
        2.0       1.00      0.03      0.07        29

avg / total       0.55      0.52      0.53      2556

-------------------- 
 categories_name {} message 
              precision    recall  f1-score   support

        0.0       0.64      0.96      0.77      1647
        1.0       0.34      0.04      0.07       909

avg / total       0.54      0.63      0.52      2556

-------------------- 
 categories_name {} original 
              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2553
        1.0       0.00      0.00      0.00         3

avg / total       1.00      1.00      1.00      2556

-------------------- 
 categories_name {} genre 
              precision    recall  f1-score   support

        0.0       0.61      0.95      0.74    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


IndexError: index 36 is out of bounds for axis 1 with size 36

In [25]:
cv2 = GridSearchCV(pipeline2, param_grid=parameters)
cv2

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect_count__ngram_range': ((1, 1), (1, 2)), 'vect_count__max_df': (0.5, 0.75, 1.0), 'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 9. Export your model as a pickle file

In [26]:
import pickle
model_filepath = 'train_model'
pickle.dump(cv, open(model_filepath, 'wb'))

In [ ]:
model=pickle.load('train_model.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.